In [179]:
import pandas as pd 
import numpy as np
import random
from Bio import pairwise2
from Bio.Seq import Seq 

In [180]:

# for efficient exact matching
def bwt(p):
    """Burrows-Wheeler transform of a string"""
    if len(p)==0:
        return None
    s='$'+p
    rotations = list(enumerate([s[i:] + s[:i] for i in range(len(s))]))
    rotations.sort(key=lambda x: x[-1])
    df = pd.DataFrame([[rot[0],rot[1][0],rot[1][-1]] for rot in rotations])
    df.columns = ['index','L','R']

    
    letters = sorted(list(df['R'].unique()))
    rank = pd.DataFrame()
    for let in letters:
        rank[let] = (df['R']==let).cumsum() - (df['R']==let)
    
    rank.loc[len(s)]=df['R'].value_counts().sort_index()

    rng = pd.DataFrame([],index=letters)
    d = df['L'].value_counts().sort_index()
    rng['start']=d.cumsum() - d
    rng['end']=d.cumsum()
    
    return df ,rank,rng



In [181]:

# Uses BWT to get the exact matching regions in the 2 sequences
def get_lmem(q,df,rank,rng):
    """Burrows-Wheeler search of a string"""
    n =len(q)
    st,end = rng.loc[q[-1]]
    j2 = n-1
    lmem_list=[]
    l_=5
    for j in range(n-2,-1,-1):
        q_ = q[j]
        #print(q_)
        st_ = rank.loc[st,q_]
        end_ = rank.loc[end,q_]
        
        if(st_==end_):
            #print('No match')
            j1 = j+1 
            l = j2-j1
            if(l>=l_-1):
                i1=df.iloc[st:end]['index'].values[0]-1
                #print(i1)
                i2 = i1 + l
                lmem_list.append((i1,i2+1,j1,j2+1,i1-j1))
            j2 = j
            st = rng.loc[q_,'start']
            end = rng.loc[q_,'end']

        elif j == 0:
            #print('Match 0')
            st = rng.loc[q_,'start'] + st_
            end = rng.loc[q_,'start'] + end_     
            j1 = j
            l = j2-j1
            if(l>=l_-1):
                i1=df.iloc[st:end]['index'].values[0]-1
                #print(i1)
                i2 = i1 + l
                lmem_list.append((i1,i2+1,j1,j2+1,i1-j1))

        else:
            st = rng.loc[q_,'start'] + st_
            end = rng.loc[q_,'start'] + end_
        
    return lmem_list
        

In [182]:

def get_colinear_sets(lmem_list):
    """Get colinear sets from lmem_list"""
    lmem_list.sort(key=lambda x: x[-1])
    thresh = 5 # for easier visualization 30
    colinear_sets = []
    l= [lmem_list[0]]
    for i in range(1,len(lmem_list)):
        if abs(lmem_list[i][-1]-lmem_list[i-1][-1])<=thresh:
            l.append(lmem_list[i])
        else:
            if(len(l)>1):
                colinear_sets.append(l)
            l = [lmem_list[i]]
    if len(l)>1:
        colinear_sets.append(l)
    return colinear_sets

In [183]:


def remove_overlap(col_set):
    for i in range(len(col_set)-1):
        lmem1 = col_set[i]
        lmem2 = col_set[i+1]
        l1 = lmem1[1]-lmem1[0]
        l2 = lmem2[1]-lmem2[0]
        if lmem1[1]>lmem2[0]:
            print("Bazinga")
            if l1>l2:
                col_set[i+1] = [lmem1[1],lmem2[1],lmem2[2],lmem2[2]+lmem2[1]-lmem1[1],lmem2[2]-lmem1[1]]
            else:
                col_set[i] = [lmem1[0],lmem2[0],lmem1[2],lmem1[2]+lmem2[0]-lmem1[0],lmem1[4]]
    return col_set


In [184]:
def non_outlier(col_set):
    col_set.sort(key=lambda x: x[2])
    thresh = 2  # for easier visualization 10
    n=len(col_set)
    dp = np.ones(n)
    for i in range(1,n):
        for j in range(i-1,-1,-1):
            if abs(col_set[i][-1]-col_set[j][-1])<=thresh:
                dp[i] = max(dp[i],dp[j]+1)
    i = dp.argmax()
    non_outlier_set = [col_set[i]]
    for j in range(i-1,-1,-1):
        if (abs(col_set[i][-1]-col_set[j][-1])<=thresh) and (dp[j]==dp[i]-1):
            non_outlier_set.append(col_set[j])
            i=j
    non_outlier_set.reverse()
    return non_outlier_set


In [185]:

def preprocess_colinear_sets(colinear_sets): 
    """Preprocess colinear sets"""
    for i in range(len(colinear_sets)):
        col_set = colinear_sets[i]
        col_set.sort(key=lambda x: x[2])
        col_set = non_outlier(col_set) #Remover outliers
        col_set = remove_overlap(col_set) #Remove overlap
        colinear_sets[i] = col_set
    return colinear_sets

        

In [186]:
#Write helper functions for gap filling and alignment here

In [187]:
p = ''.join([random.choice('ACGT') for i in range(500)]) #for easier visualization 100000
q = ''.join([random.choice('ACGT') for i in range(500)])


In [188]:
print(p)
print(q)

GAGATGTATAACCCACTAGGGATATCTAGGGACAGGAGGCCTGGCGTGTACAAATGCGCAGGTTCGGTTTGGCCTTTGTGCCAAAGATGGACAAAACCGTCGTTATAACTGTAACTGTGTGTTCAGCTGGCGTCTTAATAGCTAAGTTGCTCACATCCTCAATGGGAGGTCACTGATCTTAGCCGCGCTACGGAGCATTCAGGTTCCACTCCGGGAAAGGTCAACAAATCTATCATAACCATTGGTAAGACGAGGATTTGCTGTTCCACTGGTGCATAGTGAAAGAATATTAAAGAAATTGCTAACTCTTCGTTAGAATGTGTAATTTATCTAGGACCGAAGAATTGTGCGAATTTTGCCGGAGAAGTCGACCATCGGATAAATCATATAATGTGGAAAGGCAGATGAGGATGCAGAGACGATACGCAATCTCAAACTTCCACATTAGTCAATCCGTCCTCTGCTGAATATCATGAATGGCGCGCGGTTTACATGAACCA
CTTTTATTGGAATAGTCACCAATTTGCCAACGATGTAGACAGACACTAACTATCACTTTACTTTGCGGGTTTACCTCTCATGAATCCGGGGTACAGCCTATTACAGTTGAAGCCGTCGTAGCTGTCTAATATCGCAATCTCAGTAGCGATGCGTCTAAATTGGGTATCGGTGTTTCCTCCATTGTTTATATAACATATCCGCCCATTAGCTGCTACCCTAACACTTTACAGAGGAGTTATATATGTCACCAAAGCCTGGATCACGGGAATCCTAACGGAATCTTGCTGATGTTCATAGAGGCATTGTGGGGACTACGGCACGGACGGTGTGCCTTCTCCGGTGCCTTAAGTAAGTGACACGACCGGCGGTTAGCGACCGGTGGAACTACACATTCTTGGGTACATAGCTCGTTCGCTCCGGATCTGAACATAACCATCGGATTCTCTCTCCCACTCTCGCTCGATATGCAGCTCCAAGTTACCATCCCTGTCTAGCCGT

In [189]:
df,rank,rng = bwt(p)
df

,index,L,R
0,0,$,A
1,500,A,C
2,93,A,C
3,94,A,A
4,434,A,C
...,...,...,...
496,355,T,T
497,257,T,A
498,68,T,G
499,75,T,C


In [190]:
lmem_set = get_lmem(q,df,rank,rng)
pd.DataFrame(lmem_set,columns=['i1','i2','j1','j2','offset'])

,i1,i2,j1,j2,offset
0,370,376,480,486,-110
1,143,148,475,480,-332
2,410,416,465,471,-55
3,304,309,452,457,-148
4,370,380,432,442,-62
5,463,468,423,428,40
6,375,380,418,423,-43
7,195,200,390,395,-195
8,368,373,373,378,-5
9,311,316,368,373,-57


In [191]:
colinear_sets = get_colinear_sets(lmem_set)
colinear_sets

[[(370, 380, 432, 442, -62),
  (232, 237, 292, 297, -60),
  (311, 316, 368, 373, -57),
  (410, 416, 465, 471, -55),
  (118, 123, 169, 174, -51)],
 [(129, 135, 150, 156, -21), (96, 102, 112, 118, -16)],
 [(298, 303, 282, 287, 16), (138, 143, 118, 123, 20)],
 [(463, 468, 423, 428, 40), (343, 349, 302, 308, 41)],
 [(406, 411, 230, 235, 176), (229, 235, 49, 55, 180)],
 [(255, 261, 21, 27, 234), (442, 448, 203, 209, 239)]]

In [192]:
colinear_sets = preprocess_colinear_sets(colinear_sets)
colinear_sets

[[(232, 237, 292, 297, -60), (370, 380, 432, 442, -62)],
 [(96, 102, 112, 118, -16)],
 [(138, 143, 118, 123, 20)],
 [(343, 349, 302, 308, 41), (463, 468, 423, 428, 40)],
 [(229, 235, 49, 55, 180)],
 [(255, 261, 21, 27, 234)]]

In [193]:
#touple as in (i1,i2,j1,j2)
normal_set=[]
for similar_set in colinear_sets:
    sets=[]
    for i in range(len(similar_set)-1):
        exact_region1=similar_set[i]
        exact_region2=similar_set[i+1]
        gap_region=(exact_region1[1],exact_region2[0],exact_region1[3],exact_region2[2])
        sets.append(gap_region)
    normal_set.append(sets)

In [194]:
normal_set

[[(237, 370, 297, 432)], [], [], [(349, 463, 308, 423)], [], []]

In [198]:
similar_regions={}
normal_regions={}


In [199]:
for sim_r in colinear_sets:
    for touple in sim_r:
        similar_regions[touple[0]]=touple

In [200]:
for gap_r in normal_set:
    for touple in gap_r:
        normal_regions[touple[0]]=touple

In [201]:
final_p=""
final_q=""
alignment=""

In [202]:
position = 0
while position<len(p):
    if position in similar_regions:
        touple=similar_regions[position]
        final_p+=p[touple[0]:touple[1]]
        final_q+=q[touple[2]:touple[3]]
        alignment+="|"*(touple[1]-touple[0])
        position=touple[1]
    elif position in normal_regions:
        touple=normal_regions[position]
        tempp=p[touple[0]:touple[1]]
        tempq=q[touple[2]:touple[3]]
        seq1 = Seq(tempp) 
        seq2 = Seq(tempq)
        alignments = pairwise2.align.globalxx(seq1, seq2)
        onealignment=alignments[0]  # taking only 1 max score alignment since otherwise there could be many possiblities
        alignment_p=onealignment[0]
        alignment_q=onealignment[1]
        all=""
        for i in range(len(alignment_p)): 
            if alignment_p[i]==alignment_q[i]:
                all+="|" 
            else:
                all+=" "
        alignment+=all
        final_p+=alignment_p
        final_q+=alignment_q
        position=touple[1]
    else:
        final_p+=p[position]
        final_q+=q[position]
        position+=1
        alignment+=" "

In [205]:
print(final_p[:200])
print(alignment[:200])
print(final_q[:200])

GAGATGTATAACCCACTAGGGATATCTAGGGACAGGAGGCCTGGCGTGTACAAATGCGCAGGTTCGGTTTGGCCTTTGTGCCAAAGATGGACAAAACCGTCGTTATAACTGTAACTGTGTGTTCAGCTGGCGTCTTAATAGCTAAGTTGCTCACATCCTCAATGGGAGGTCACTGATCTTAGCCGCGCTACGGAGCATTC
                                                                                                ||||||                                    |||||                                                         
CTTTTATTGGAATAGTCACCAATTTGCCAACGATGTAGACAGACACTAACTATCACTTTACTTTGCGGGTTTACCTCTCATGAATCCGGGGTACAGCCGTCGACAGTTGAAGCCGTCGTAGCTGTCTAATATCGCAATTAGCTTAGCGATGCGTCTAAATTGGGTATCGGTGTTTCCTCCATTGTTTATATAACATATCC


In [ ]:
q
